In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
items = pd.read_csv('order_items.csv')[['id','order_id','product_id','quantity','created_at']]

In [3]:
items['created_at']=pd.to_datetime(items['created_at'])

Transforming datestamp to MM/YY to be suitable for cohorts

In [4]:
items['created_at_month_year']= items['created_at'].map(lambda x: x.strftime('%Y-%m'))

In [5]:
orders = pd.read_csv('orders.csv')[['id','customer_id','created_at']]

In [6]:
orders['created_at'] = pd.to_datetime(orders['created_at'])

In [7]:
orders['created_at_month_year']= orders['created_at'].map(lambda x: x.strftime('%Y-%m'))

In [8]:
cohort = orders.sort_values(by='created_at_month_year').groupby('customer_id')['created_at_month_year'].first()

In [9]:
cohort = cohort.to_frame(name='cohort') #need to transform the above pandas series to df

In [10]:
cohort.head()

,cohort
customer_id,
1,2013-04
2,2013-04
3,2013-04
4,2013-04
5,2013-04


#### We cannot join Orders and Order Items, as timestamps differ by two years. Also no foreign keys available, id, order_id differ 
see below

In [11]:
items.head()

,id,order_id,product_id,quantity,created_at,created_at_month_year
0,1,1,1,1,2015-04-02 12:00:00,2015-04
1,2,2,6,1,2015-04-08 12:00:00,2015-04
2,3,3,2,1,2015-04-05 12:00:00,2015-04
3,4,4,1,1,2015-04-06 12:00:00,2015-04
4,5,5,1,1,2015-04-02 12:00:00,2015-04


In [12]:
orders.head()

,id,customer_id,created_at,created_at_month_year
0,1,1,2013-04-07 12:00:00,2013-04
1,2,2,2013-04-04 12:00:00,2013-04
2,3,4,2013-04-06 12:00:00,2013-04
3,4,3,2013-04-08 12:00:00,2013-04
4,5,10,2013-04-04 12:00:00,2013-04


In [13]:
products = pd.read_csv('products.csv')[['id','price','created_at']]

In [14]:
products['created_at']=pd.to_datetime(products['created_at'])

In [15]:
products['created_at_month_year']= products['created_at'].map(lambda x: x.strftime('%Y-%m'))

In [16]:
items.describe()

,id,order_id,product_id,quantity
count,665870.00000,665870.00000,665870.000000,665870.0
mean,332935.50000,332935.50000,5.496139,1.0
std,192220.25621,192220.25621,2.872268,0.0
min,1.00000,1.00000,1.000000,1.0
25%,166468.25000,166468.25000,3.000000,1.0
50%,332935.50000,332935.50000,5.000000,1.0
75%,499402.75000,499402.75000,8.000000,1.0
max,665870.00000,665870.00000,10.000000,1.0


In [17]:
products.head(1)

,id,price,created_at,created_at_month_year
0,1,8999,2016-04-24 19:04:05,2016-04


In [18]:
type(products['price'])

pandas.core.series.Series

In [43]:
df_items = pd.merge(items[['product_id','order_id', 'quantity']],products[['id','price']],
                    how = 'left', left_on='product_id',right_on='id')

In [44]:
df_items.head(2)

,product_id,order_id,quantity,id,price
0,1,1,1,1,8999
1,6,2,1,6,7999


In [29]:
assert len(items) == len(df_items)

In [45]:
df_items['revenue']=df_items['quantity'] * df_items['price']

In [23]:
#cohort_short = cohort[cohort['cohort']>='2015-04']

In [24]:
#cohort_short.head(1)

,cohort
customer_id,
724060,2015-04


In [46]:
df_orders = pd.merge(df_items[['order_id','revenue']],orders[['customer_id','id']],
                     how = 'left', left_on='order_id',right_on='id')

In [47]:
df_orders.head()

,order_id,revenue,customer_id,id
0,1,8999,1,1
1,2,7999,2,2
2,3,4999,4,3
3,4,8999,3,4
4,5,8999,10,5


In [48]:
df_cohorts = pd.merge(df_orders[['customer_id', 'revenue']],
                      cohort.reset_index()[['customer_id','cohort']],
                      how = 'left', on='customer_id')

In [50]:
df_cohorts = pd.merge(df_cohorts[['customer_id', 'revenue','cohort']],
                      orders[['customer_id','created_at_month_year']],
                      how = 'left', on='customer_id')

In [51]:
df_cohorts.head()

,customer_id,revenue,cohort,created_at_month_year
0,1,8999,2013-04,2013-04
1,1,8999,2013-04,2013-10
2,1,8999,2013-04,2015-03
3,1,8999,2013-04,2015-04
4,2,7999,2013-04,2013-04


We made two assumptions: Id in Orders corresponds to order_id in order_items
and 
Cohort periods are based on orders.

In [52]:
cohort_pivot1 = pd.pivot_table(df_cohorts, values = 'revenue', 
               index = 'created_at_month_year', columns = 'cohort', aggfunc = 'sum')

In [53]:
cohort_pivot1

cohort,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04
created_at_month_year,,,,,,,,,,,,,
2013-04,528091201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05,87876474.0,539915881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-06,88224179.0,89532341.0,496914537.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07,90849810.0,95763461.0,87816593.0,483795773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-08,94287409.0,95049098.0,90334131.0,86618977.0,453766439.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-09,92121400.0,93762416.0,85817966.0,86102889.0,79203548.0,409048150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10,93022845.0,96106599.0,88156095.0,90321119.0,83221979.0,74283380.0,396939286.0,NaN,NaN,NaN,NaN,NaN,NaN
2013-11,90668871.0,91902178.0,86221062.0,87933195.0,83317335.0,73703816.0,71575602.0,357513692.0,NaN,NaN,NaN,NaN,NaN
2013-12,92331510.0,97257771.0,89710135.0,87217236.0,85169846.0,77797731.0,76270662.0,68816850.0,341575182.0,NaN,NaN,NaN,NaN


In [54]:
cohort_pivot1.to_csv('Revenue_Cohorts.csv')